In [9]:
import pandas as pd

### Загрузка данных

In [10]:
raw_data = pd.read_csv("NASDAQ_FULL_2021_Close.csv")
raw_data.head()

,Date,AAPL,ABBV,ABT,ACN,ADBE,ADP,AMD,AMGN,AMT,...,HLAHU,HLAHW,HLBZW,HLGN-WT,HLLY-WT,HMA-UN,HMA-WT,HMACR,HMACU,HMACW
0,2021-01-04,125.974464,86.739708,99.687309,238.710358,485.339996,153.288055,92.300003,193.642502,191.466125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-05,127.532021,87.636642,100.920723,240.069351,485.690002,152.861603,92.769997,194.582230,192.433868,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-06,123.239037,86.879608,100.710602,242.694153,466.309998,152.326324,90.330002,199.272491,187.359482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-07,127.444374,87.809464,101.688164,244.983932,477.739990,153.188248,95.160004,199.930344,189.635117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-08,128.544388,88.270248,101.971436,245.877502,485.100006,155.184143,94.580002,203.749222,193.733017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Фильтрация активов

In [11]:
raw_data.isna().sum(axis=0).value_counts().sort_index()

0      6325
1        13
2         3
4        72
5         4
       ... 
246       1
247       1
248       2
250       3
252    4727
Name: count, Length: 220, dtype: int64

Удалим те, что имеют более 50 пропусков (n/a). Это те активы, по которым либо не нашлось данных (ещё не обращались на бирже), либо листинг которых произошёл в 2021 году (например, в его середине), и полных данных по ним нет.

In [12]:
MAX_NA_THRESHOLD = 50

In [13]:
na_counts = raw_data.isna().sum(axis=0)
valid_tickers = na_counts[na_counts <= MAX_NA_THRESHOLD].index

data = raw_data[valid_tickers]

In [14]:
print(f"Tickers was: {data.shape[1]}")
print(f"Deleted tickers (NA > {MAX_NA_THRESHOLD}): {raw_data.shape[1] - data.shape[1]}")
print(f"Tickers left for analysis: {data.shape[1]}")

Tickers was: 6562
Deleted tickers (NA > 50): 5289
Tickers left for analysis: 6562


### Заполнение пропусков

In [15]:
data.ffill(inplace=True)
data.bfill(inplace=True)

C:\Users\Yuriy\AppData\Local\Temp\ipykernel_32408\2526927714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.ffill(inplace=True)
C:\Users\Yuriy\AppData\Local\Temp\ipykernel_32408\2526927714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.bfill(inplace=True)


Проверим, что мы точно почистили все n/a:

In [18]:
data.isna().sum(axis=0).value_counts()

0    6562
Name: count, dtype: int64

Экспортируем очищенные данные:

In [19]:
data.to_csv("data/processed/NASDAQ_FULL_2021_Cleaned_Imputed.csv")